In [1]:
import pandas as pd
from pandas import DataFrame, Series
import sqlite3 as db

url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
col_list = ['age','workclass','fnlwgt','education','education-num','marital-status','occupation',
           'relationship','race','sex','capital-gain','capital-loss','hours-per-week','native-country','Label']
# Import the data from the url above into Pandas DataFrame
adult = pd.read_csv(url,sep=",",delimiter=",",names=col_list,skipinitialspace=True)

print(adult.columns)
# import regular expression library re.
# this is done in order to identify the column names with - in them and convert them to underscore "_" as hypen " - "
# is not a valid character in database
import re
adult.columns = [re.sub("[-]", "_", col) for col in adult.columns]

print(adult.columns)

# print first five rows of the dataframe head to chcek how the data loaded looks like
adult.head()

Index(['age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'Label'],
      dtype='object')
Index(['age', 'workclass', 'fnlwgt', 'education', 'education_num',
       'marital_status', 'occupation', 'relationship', 'race', 'sex',
       'capital_gain', 'capital_loss', 'hours_per_week', 'native_country',
       'Label'],
      dtype='object')


,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,Label
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [2]:
#1. Create an sqlalchemy engine using a sample from the data set
# import corresponding sqlalchemy library
from sqlalchemy import *
from sqlalchemy import create_engine, ForeignKey
from sqlalchemy import Column, Date, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, backref

#create database engine
engine = create_engine('sqlite://', echo=True)

#connect to database and load  the dataframe adultdb into sqllite 
adult.to_sql('adultdb', con=engine)

2019-04-09 08:54:14,029 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-04-09 08:54:14,030 INFO sqlalchemy.engine.base.Engine ()
2019-04-09 08:54:14,032 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-04-09 08:54:14,032 INFO sqlalchemy.engine.base.Engine ()
2019-04-09 08:54:14,034 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("adultdb")
2019-04-09 08:54:14,040 INFO sqlalchemy.engine.base.Engine ()
2019-04-09 08:54:14,045 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE adultdb (
	"index" BIGINT, 
	age BIGINT, 
	workclass TEXT, 
	fnlwgt BIGINT, 
	education TEXT, 
	education_num BIGINT, 
	marital_status TEXT, 
	occupation TEXT, 
	relationship TEXT, 
	race TEXT, 
	sex TEXT, 
	capital_gain BIGINT, 
	capital_loss BIGINT, 
	hours_per_week BIGINT, 
	native_country TEXT, 
	"Label" TEXT
)


2019-04-09 08:54:14,046 INFO sqlalchemy.engine.base.Engine ()
2019-04-09 08:54:14,049 INFO sqlal

In [3]:
from sqlalchemy.orm import Session
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker


# create an Object to hold SQLAlchemy data types to map properties of Python classes into columns on a relation database table
Base = declarative_base(engine)
class Adultdb(Base):
    """
    eg. fields: id, title
    """
    __tablename__ = 'adultdb'
    __table_args__ = {'autoload': True}
    
    index = Column(Integer(), primary_key=True)
    age = Column(Integer())
    workclass = Column(String())

#Create SQLAlchemy Sessions

def loadSession():
    """"""
    metadata = Base.metadata
    Session = sessionmaker(bind=engine)
    session = Session()
    return session

#Querying Data with SQLAlchemy ORM
if __name__ == "__main__":
    session = loadSession()
    rows = session.query(Adultdb).first()
    print("AGE","SEX   ","WORKCLAS ","COUNTRY ","OCCUPATION")
    print("*"*80)
    print(rows.age,rows.sex,rows.workclass,rows.native_country,rows.occupation)

2019-04-09 08:54:46,678 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("adultdb")
2019-04-09 08:54:46,680 INFO sqlalchemy.engine.base.Engine ()
2019-04-09 08:54:46,682 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'adultdb' AND type = 'table'
2019-04-09 08:54:46,682 INFO sqlalchemy.engine.base.Engine ()
2019-04-09 08:54:46,683 INFO sqlalchemy.engine.base.Engine PRAGMA foreign_key_list("adultdb")
2019-04-09 08:54:46,690 INFO sqlalchemy.engine.base.Engine ()
2019-04-09 08:54:46,693 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'adultdb' AND type = 'table'
2019-04-09 08:54:46,696 INFO sqlalchemy.engine.base.Engine ()
2019-04-09 08:54:46,703 INFO sqlalchemy.engine.base.Engine PRAGMA index_list("adultdb")
2019-04-09 08:54:46,704 INFO sqlalchemy.engine.base.Engine ()
2019-04-09 08:54:46,705 INFO sqlal

In [4]:
#2. Write two basic update queries
if __name__ == "__main__":
    session = loadSession()
    rows = session.query(Adultdb).filter_by(fnlwgt=77516).first()
    print(rows)
    rows.occupation ='Student'
    rows.workclass = 'Study'
    session.commit()
    
#verify update results
if __name__ == "__main__":
    session = loadSession()
    rows = session.query(Adultdb).filter_by(fnlwgt=77516).first()
    print("occupation : ",rows.occupation,"workclass :",rows.workclass,"fnlwgt : ",rows.fnlwgt)     

2019-04-09 08:55:50,912 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-04-09 08:55:50,915 INFO sqlalchemy.engine.base.Engine SELECT adultdb."index" AS adultdb_index, adultdb.age AS adultdb_age, adultdb.workclass AS adultdb_workclass, adultdb.fnlwgt AS adultdb_fnlwgt, adultdb.education AS adultdb_education, adultdb.education_num AS adultdb_education_num, adultdb.marital_status AS adultdb_marital_status, adultdb.occupation AS adultdb_occupation, adultdb.relationship AS adultdb_relationship, adultdb.race AS adultdb_race, adultdb.sex AS adultdb_sex, adultdb.capital_gain AS adultdb_capital_gain, adultdb.capital_loss AS adultdb_capital_loss, adultdb.hours_per_week AS adultdb_hours_per_week, adultdb.native_country AS adultdb_native_country, adultdb."Label" AS "adultdb_Label" 
FROM adultdb 
WHERE adultdb.fnlwgt = ?
 LIMIT ? OFFSET ?
2019-04-09 08:55:50,915 INFO sqlalchemy.engine.base.Engine (77516, 1, 0)
2019-04-09 08:55:50,918 INFO sqlalchemy.engine.base.Engine UPDATE adultdb SET wo

In [6]:
#update query 2 
if __name__ == "__main__":
    session = loadSession()
    row = session.query(Adultdb).filter_by(age=60,education='Masters').all()
    for i in row:
        i.occupation = 'Retired'
    session.commit()
    
if __name__ == "__main__":
    session = loadSession()
    row = session.query(Adultdb).filter_by(age=60,education='Masters').all()
    for i in row:
        print(i.age,i.education,i.sex,i.occupation,i.workclass)    

2019-04-09 08:56:25,822 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-04-09 08:56:25,829 INFO sqlalchemy.engine.base.Engine SELECT adultdb."index" AS adultdb_index, adultdb.age AS adultdb_age, adultdb.workclass AS adultdb_workclass, adultdb.fnlwgt AS adultdb_fnlwgt, adultdb.education AS adultdb_education, adultdb.education_num AS adultdb_education_num, adultdb.marital_status AS adultdb_marital_status, adultdb.occupation AS adultdb_occupation, adultdb.relationship AS adultdb_relationship, adultdb.race AS adultdb_race, adultdb.sex AS adultdb_sex, adultdb.capital_gain AS adultdb_capital_gain, adultdb.capital_loss AS adultdb_capital_loss, adultdb.hours_per_week AS adultdb_hours_per_week, adultdb.native_country AS adultdb_native_country, adultdb."Label" AS "adultdb_Label" 
FROM adultdb 
WHERE adultdb.age = ? AND adultdb.education = ?
2019-04-09 08:56:25,831 INFO sqlalchemy.engine.base.Engine (60, 'Masters')
2019-04-09 08:56:25,855 INFO sqlalchemy.engine.base.Engine COMMIT
2019-04

In [7]:
#3. Write two delete queries
#Check for all rows which have occupation as ? - Querying Data with SQLAlchemy ORM
if __name__ == "__main__":
    session = loadSession()
    rows = session.query(Adultdb).filter_by(occupation="?").all()
    print("Count of rows before delete operation : ",len(rows))

#Delete rows which have occupation as "?" - Delete Data with SQLAlchemy ORM
if __name__ == "__main__":
    session = loadSession()
    session.query(Adultdb).filter_by(occupation="?").delete(synchronize_session='fetch')
    session.commit()
    rows = session.query(Adultdb).filter_by(occupation="?").all()
    print("Count of rows after delete operation : ",len(rows))    

2019-04-09 08:57:19,684 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-04-09 08:57:19,690 INFO sqlalchemy.engine.base.Engine SELECT adultdb."index" AS adultdb_index, adultdb.age AS adultdb_age, adultdb.workclass AS adultdb_workclass, adultdb.fnlwgt AS adultdb_fnlwgt, adultdb.education AS adultdb_education, adultdb.education_num AS adultdb_education_num, adultdb.marital_status AS adultdb_marital_status, adultdb.occupation AS adultdb_occupation, adultdb.relationship AS adultdb_relationship, adultdb.race AS adultdb_race, adultdb.sex AS adultdb_sex, adultdb.capital_gain AS adultdb_capital_gain, adultdb.capital_loss AS adultdb_capital_loss, adultdb.hours_per_week AS adultdb_hours_per_week, adultdb.native_country AS adultdb_native_country, adultdb."Label" AS "adultdb_Label" 
FROM adultdb 
WHERE adultdb.occupation = ?
2019-04-09 08:57:19,692 INFO sqlalchemy.engine.base.Engine ('?',)
Count of rows before delete operation :  1841
2019-04-09 08:57:19,740 INFO sqlalchemy.engine.base.Eng

In [8]:

#Delete 2
#Check for all rows which have education as  Some-college- Querying Data with SQLAlchemy ORM
if __name__ == "__main__":
    session = loadSession()
    rows = session.query(Adultdb).filter_by(education="Some-college").all()
    print("count of rows :",len(rows))
    
#delete rows from table adultdb which have education as "Some-college" - Delete Data with SQLAlchemy ORM
#delete(synchronize_session='fetch')
#'fetch' - performs a select query before the delete to find objects that are matched by the delete query and 
#need to be removed from the session. Matched objects are removed from the session
if __name__ == "__main__":
    session = loadSession()
    session.query(Adultdb).filter_by(education="Some-college").delete(synchronize_session='fetch')
    session.commit()
    rows = session.query(Adultdb).filter_by(education="Some-college").all()
    print("Count of rows after Delete : ",len(rows))    

2019-04-09 08:57:43,170 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-04-09 08:57:43,174 INFO sqlalchemy.engine.base.Engine SELECT adultdb."index" AS adultdb_index, adultdb.age AS adultdb_age, adultdb.workclass AS adultdb_workclass, adultdb.fnlwgt AS adultdb_fnlwgt, adultdb.education AS adultdb_education, adultdb.education_num AS adultdb_education_num, adultdb.marital_status AS adultdb_marital_status, adultdb.occupation AS adultdb_occupation, adultdb.relationship AS adultdb_relationship, adultdb.race AS adultdb_race, adultdb.sex AS adultdb_sex, adultdb.capital_gain AS adultdb_capital_gain, adultdb.capital_loss AS adultdb_capital_loss, adultdb.hours_per_week AS adultdb_hours_per_week, adultdb.native_country AS adultdb_native_country, adultdb."Label" AS "adultdb_Label" 
FROM adultdb 
WHERE adultdb.education = ?
2019-04-09 08:57:43,178 INFO sqlalchemy.engine.base.Engine ('Some-college',)
count of rows : 6775
2019-04-09 08:57:43,382 INFO sqlalchemy.engine.base.Engine BEGIN (impl

In [9]:
#4. Write two filter queries
# Querying Data with SQLAlchemy ORM based on filter on workclass column having a value as "Private"
if __name__ == "__main__":
    session = loadSession()
    rows = session.query(Adultdb).filter_by(workclass='Private').all()
    for i in rows:
        print(i.age,i.sex,i.workclass,i.native_country)
        
# Querying Data with SQLAlchemy ORM based on filter on native_country column having a value as "Columbia"
count = 0
for rows in engine.execute("SELECT age,sex,occupation FROM adultdb where native_country = 'Columbia' ").fetchall():
    print(rows.age," ",rows.sex," ",rows.occupation)
    count+=1
print("Total Records Retrieved : ",count)        

2019-04-09 08:58:39,561 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-04-09 08:58:39,563 INFO sqlalchemy.engine.base.Engine SELECT adultdb."index" AS adultdb_index, adultdb.age AS adultdb_age, adultdb.workclass AS adultdb_workclass, adultdb.fnlwgt AS adultdb_fnlwgt, adultdb.education AS adultdb_education, adultdb.education_num AS adultdb_education_num, adultdb.marital_status AS adultdb_marital_status, adultdb.occupation AS adultdb_occupation, adultdb.relationship AS adultdb_relationship, adultdb.race AS adultdb_race, adultdb.sex AS adultdb_sex, adultdb.capital_gain AS adultdb_capital_gain, adultdb.capital_loss AS adultdb_capital_loss, adultdb.hours_per_week AS adultdb_hours_per_week, adultdb.native_country AS adultdb_native_country, adultdb."Label" AS "adultdb_Label" 
FROM adultdb 
WHERE adultdb.workclass = ?
2019-04-09 08:58:39,565 INFO sqlalchemy.engine.base.Engine ('Private',)
38 Male Private United-States
53 Male Private United-States
28 Female Private Cuba
37 Female Pri

34 Male Private United-States
50 Male Private United-States
42 Male Private United-States
44 Female Private Puerto-Rico
26 Male Private United-States
36 Male Private United-States
41 Female Private United-States
67 Male Private United-States
57 Female Private United-States
29 Male Private United-States
29 Male Private United-States
27 Male Private United-States
32 Female Private United-States
58 Female Private United-States
21 Male Private United-States
72 Female Private United-States
35 Female Private United-States
69 Female Private United-States
35 Male Private United-States
31 Female Private ?
34 Female Private United-States
30 Male Private Mexico
28 Male Private United-States
54 Male Private United-States
24 Female Private United-States
20 Male Private United-States
45 Female Private Mexico
47 Male Private United-States
24 Male Private United-States
51 Male Private United-States
17 Male Private United-States
37 Male Private United-States
27 Female Private United-States
29 Male Priv

59 Female Private United-States
32 Male Private United-States
28 Male Private Mexico
47 Male Private United-States
36 Male Private United-States
38 Male Private United-States
50 Male Private United-States
25 Male Private United-States
60 Male Private United-States
46 Male Private United-States
56 Female Private United-States
25 Male Private United-States
46 Male Private United-States
39 Male Private United-States
40 Male Private United-States
49 Female Private United-States
24 Male Private United-States
40 Male Private United-States
61 Male Private United-States
25 Male Private United-States
47 Male Private United-States
38 Female Private United-States
26 Female Private United-States
17 Female Private United-States
58 Male Private United-States
26 Male Private United-States
31 Male Private Mexico
30 Male Private United-States
25 Female Private United-States
37 Male Private United-States
26 Female Private United-States
45 Male Private United-States
27 Male Private United-States
32 Male 

28 Male Private United-States
22 Male Private United-States
33 Male Private United-States
25 Male Private United-States
37 Male Private Mexico
55 Male Private England
20 Female Private United-States
43 Female Private United-States
54 Male Private England
17 Male Private United-States
63 Female Private United-States
37 Male Private United-States
27 Male Private United-States
22 Male Private United-States
20 Male Private United-States
25 Male Private United-States
53 Female Private United-States
19 Female Private United-States
34 Male Private United-States
70 Male Private United-States
57 Male Private United-States
34 Male Private United-States
42 Female Private United-States
45 Male Private United-States
51 Male Private Mexico
44 Male Private United-States
35 Male Private United-States
68 Female Private United-States
29 Male Private United-States
61 Male Private United-States
34 Male Private United-States
41 Female Private United-States
60 Male Private United-States
35 Male Private Chin

24 Male Private United-States
40 Female Private United-States
45 Female Private United-States
27 Female Private United-States
36 Male Private Mexico
30 Male Private Germany
36 Female Private United-States
25 Male Private Mexico
31 Female Private United-States
49 Male Private United-States
26 Male Private United-States
25 Male Private United-States
29 Female Private United-States
41 Male Private United-States
35 Male Private United-States
21 Female Private Mexico
29 Female Private United-States
51 Male Private United-States
39 Male Private United-States
36 Male Private United-States
61 Male Private United-States
38 Male Private United-States
21 Female Private United-States
40 Female Private United-States
46 Male Private United-States
39 Male Private United-States
30 Female Private United-States
28 Male Private Mexico
32 Male Private United-States
40 Male Private United-States
47 Female Private United-States
44 Male Private United-States
38 Male Private Columbia
51 Male Private Iran
45 M

...

42 Male Private United-States
25 Female Private United-States
31 Female Private United-States
48 Female Private United-States
40 Male Private United-States
28 Female Private United-States
48 Female Private Trinadad&Tobago
19 Female Private United-States
59 Male Private United-States
32 Male Private United-States
39 Male Private United-States
33 Male Private United-States
38 Male Private United-States
23 Female Private United-States
31 Male Private United-States
24 Female Private United-States
59 Male Private United-States
22 Male Private United-States
20 Male Private El-Salvador
20 Male Private United-States
27 Male Private United-States
17 Female Private United-States
60 Male Private United-States
30 Female Private United-States
24 Male Private United-States
23 Female Private United-States
24 Male Private United-States
48 Male Private United-States
31 Male Private United-States
33 Male Private United-States
30 Male Private United-States
38 Male Private United-States
58 Female Private 

48 Male Private United-States
46 Male Private ?
33 Male Private United-States
33 Male Private United-States
47 Male Private Mexico
40 Female Private United-States
42 Male Private Mexico
51 Male Private United-States
37 Female Private United-States
25 Female Private United-States
52 Male Private United-States
28 Female Private United-States
30 Male Private United-States
34 Male Private United-States
31 Male Private United-States
38 Male Private United-States
18 Male Private United-States
26 Female Private United-States
36 Male Private United-States
43 Male Private United-States
26 Male Private United-States
18 Female Private United-States
41 Male Private United-States
31 Female Private United-States
56 Female Private United-States
50 Male Private United-States
22 Female Private United-States
48 Male Private India
30 Female Private United-States
56 Female Private United-States
41 Male Private United-States
34 Female Private Dominican-Republic
42 Female Private United-States
33 Male Priva

43 Female Private United-States
20 Male Private United-States
20 Male Private United-States
42 Male Private United-States
34 Male Private United-States
43 Male Private United-States
33 Male Private Japan
47 Male Private United-States
41 Male Private Mexico
40 Male Private United-States
28 Male Private United-States
29 Female Private United-States
28 Male Private United-States
24 Female Private United-States
22 Male Private United-States
18 Male Private United-States
41 Male Private ?
36 Female Private United-States
21 Male Private United-States
46 Male Private United-States
25 Female Private India
17 Male Private United-States
45 Male Private United-States
30 Male Private United-States
40 Male Private United-States
42 Male Private United-States
53 Male Private United-States
37 Male Private Cuba
36 Female Private United-States
46 Male Private United-States
32 Male Private United-States
17 Male Private United-States
29 Male Private United-States
54 Male Private United-States
38 Male Priv

35 Male Private ?
27 Male Private Mexico
27 Male Private United-States
31 Male Private United-States
45 Male Private Mexico
47 Female Private United-States
22 Male Private United-States
24 Female Private United-States
45 Male Private United-States
31 Female Private United-States
48 Female Private United-States
41 Female Private Haiti
34 Male Private United-States
20 Female Private United-States
39 Male Private Philippines
28 Male Private United-States
41 Male Private United-States
24 Male Private United-States
23 Female Private United-States
23 Female Private United-States
54 Male Private United-States
48 Female Private Philippines
38 Male Private United-States
22 Male Private El-Salvador
42 Male Private United-States
31 Male Private China
27 Female Private United-States
57 Male Private United-States
24 Male Private United-States
38 Male Private Mexico
29 Male Private Yugoslavia
38 Male Private United-States
61 Female Private United-States
42 Female Private United-States
21 Male Privat

23 Female Private United-States
31 Female Private United-States
47 Female Private United-States
61 Female Private United-States
35 Male Private United-States
23 Female Private Mexico
41 Female Private United-States
51 Female Private United-States
36 Male Private Mexico
34 Female Private United-States
25 Male Private United-States
37 Male Private United-States
19 Male Private United-States
23 Male Private United-States
18 Female Private United-States
51 Male Private United-States
61 Female Private United-States
53 Female Private United-States
17 Male Private United-States
61 Male Private United-States
38 Male Private United-States
33 Male Private United-States
36 Male Private United-States
85 Male Private Poland
62 Female Private United-States
24 Female Private United-States
66 Female Private United-States
37 Male Private United-States
55 Male Private United-States
50 Female Private United-States
34 Male Private United-States
36 Male Private United-States
22 Male Private United-States
3

In [10]:
#5. Write two function queries
# Querying Data with SQLAlchemy ORM - Use aggregate Function -count the number of male and female based on sex data column values
# import func module from sqlalchemy.sql package in order to perform aggregate functions
from sqlalchemy.sql import func
if __name__ == "__main__":
    session = loadSession()
    result = session.query(func.count(Adultdb.age).label('count_age'), Adultdb.sex ).group_by(Adultdb.sex).all()
    print("Count of people based on sex ")
    print('*'*80)
          
    for rows in result:
        print(rows)
        
# Querying Data with SQLAlchemy ORM - Use aggregate Function -average age and minimum age of people based on sex column value .
if __name__ == "__main__":
    session = loadSession()
    result = session.query(func.avg(Adultdb.age).label('avg_age'),func.min(Adultdb.age).label('min_age'), Adultdb.sex).group_by(Adultdb.sex).all()
    print("Average age,minimum age  of people based on sex ")
    print('*'*80)
    for rows in result:
        print(rows)        

        

2019-04-09 09:00:05,386 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-04-09 09:00:05,391 INFO sqlalchemy.engine.base.Engine SELECT count(adultdb.age) AS count_age, adultdb.sex AS adultdb_sex 
FROM adultdb GROUP BY adultdb.sex
2019-04-09 09:00:05,392 INFO sqlalchemy.engine.base.Engine ()
Count of people based on sex 
********************************************************************************
(7391, 'Female')
(16554, 'Male')
2019-04-09 09:00:05,417 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-04-09 09:00:05,418 INFO sqlalchemy.engine.base.Engine SELECT avg(adultdb.age) AS avg_age, min(adultdb.age) AS min_age, adultdb.sex AS adultdb_sex 
FROM adultdb GROUP BY adultdb.sex
2019-04-09 09:00:05,419 INFO sqlalchemy.engine.base.Engine ()
Average age,minimum age  of people based on sex 
********************************************************************************
(37.77864970910567, 17, 'Female')
(39.699287181345895, 17, 'Male')
